In [1]:
from tqdm import tqdm, tqdm_notebook
import pickle

import pandas as pd
import numpy as np

import spacy
from gensim.models import Phrases
from gensim import models,corpora

from pathlib import Path
from multiprocessing import Pool

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [3]:
PATH = Path('data/')
list(PATH.iterdir())

[PosixPath('data/train.tsv'),
 PosixPath('data/test.tsv'),
 PosixPath('data/sample_submission.csv.7z')]

In [4]:
train_df = pd.read_csv(PATH/'train.tsv',sep = '\t')

## Processing Descriptions

In [5]:
nlp = spacy.load('en')

In [23]:
def tokenize_text(df_col):
    text_corpus = df_col.apply(lambda x: str(x).lower()).values
    removal=['ADV', 'PRON', 'CCONJ', 'PUNCT', 'PART', 'DET',
             'ADP','SPACE','VERB']
    text_out_final = []

    for review in tqdm_notebook(nlp.pipe(text_corpus, n_threads=8, batch_size=1000),
                                desc='Tokenizing'):
        text_out = []
        for token in review:
            if token.pos_ not in removal and token.is_punct == False and token.is_stop == False:
                lemma = token.lemma_
                if lemma != '-PRON-':
                    text_out.append(lemma)

        text_out_final.append(text_out)
    return text_out_final

In [24]:
tokenized_text = tokenize_text(train_df['item_description'])

HBox(children=(IntProgress(value=1, bar_style='info', description='Tokenizing', max=1), HTML(value='')))

KeyboardInterrupt: 

,item_description
0,No description yet
1,This keyboard is in great condition and works ...
2,Adorable top with a hint of lace and a key hol...
3,New with tags. Leather horses. Retail for [rm]...
4,Complete with certificate of authenticity
5,"Banana republic bottoms, Candies skirt with ma..."
6,Size small but straps slightly shortened to fi...
7,You get three pairs of Sophie cheer shorts siz...
8,Girls Size small Plus green. Three shorts total.
9,I realized his pants are on backwards after th...
